In [270]:
import ipywidgets as widgets
from IPython.display import display

import matplotlib.pyplot as plt
import numpy as np
import CoolProp
import numpy as np
import math
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix, linalg
import sys
import iapws

pltp,pltt,pltq,zb,z=None,None,None,None,None

In [276]:
#l1=widgets.Label(value="Fluid Properties",style={'description_width': '200px'})
hl1 = widgets.HTML(
    value="<div style='text-align:left; font-weight:bold; font-size:24px; color:red;'>Fluid Properties</div>"
)

hl2 = widgets.HTML(
    value="<div style='text-align:left; font-weight:bold; font-size:24px; color:red;'>Well Properties</div>"
)

hl3 = widgets.HTML(
    value="<div style='text-align:left; font-weight:bold; font-size:24px; color:red;'>Time Step Control</div>"
)

hl4 = widgets.HTML(
    value="<div style='text-align:left; font-weight:bold; font-size:24px; color:red;'>Convergence Control</div>"
)




pin = widgets.FloatText(
    value=50,
    description='Pin (MPa)',
    style={'description_width': '200px'}
)

tin = widgets.FloatText(
    value=413,
    description='Tin (K)',
    style={'description_width': '200px'}
)

tsc = widgets.FloatText(
    value=298,
    description='Tsc (K)',
    style={'description_width': '200px'}
)

qsc = widgets.FloatText(
    value=0.05*86400,
    description='qsc (m3/s)',
    style={'description_width': '200px'}
)

wCO2 = widgets.FloatText(
    value=0.02,
    description='CO2 weight %',
    style={'description_width': '200px'}
)


rw = widgets.FloatText(
    value=0.23,
    description='well radius (m)',
    style={'description_width': '200px'}
)

ang = widgets.FloatText(
    value=90,
    description='inclination angle (degrees)',
    style={'description_width': '200px'}
)

nz = widgets.FloatText(
    value=100,
    description='number of cells',
    style={'description_width': '200px'}
)

h0 = widgets.FloatText(
    value=35,
    description='cell height (m)',
    style={'description_width': '200px'}
)

fdt = widgets.FloatText(
    value=1,
    description='First time step (s)',
    style={'description_width': '200px'}
)

ndt = widgets.FloatText(
    value=300,
    description='Number of timesteps',
    style={'description_width': '200px'}
)

tf = widgets.FloatText(
    value=24,
    description='Final time (hrs)',
    style={'description_width': '200px'}
)

U = widgets.FloatText(
    value=0,
    description='overall heat transfer (J/s.m2)',
    style={'description_width': '200px'}
)

tolp = widgets.FloatText(
    value=0.01,
    description='tolerance for pressure',
    style={'description_width': '200px'}
)

tolt = widgets.FloatText(
    value=0.01,
    description='tolerance for temperature',
    style={'description_width': '200px'}
)

tolq = widgets.FloatText(
    value=0.01,
    description='tolerance for flow rate',
    style={'description_width': '200px'}
)

max_allowed_dp = widgets.FloatText(
    value=100,
    description='max allowed ∆p (kPa)',
    style={'description_width': '200px'}
)

max_allowed_dt = widgets.FloatText(
    value=1,
    description='max allowed ∆T (K)',
    style={'description_width': '200px'}
)

checkbox_log = widgets.Checkbox(
    value=False,  
    description='Show log', 
    disabled=False
)

checkbox_maxAllowed = widgets.Checkbox(
    value=False,  
    description='max change in pres/temp', 
    disabled=False
)

# Button
run_button = widgets.Button(
    description='Run',
    button_style='primary'
)

# Button
plot_button = widgets.Button(
    description='Plot',
    button_style='primary'
)



# Output area
output = widgets.Output()


In [281]:
def run_calculation(button):
    with output:
        output.clear_output()

        
        
        
        ################################### 
        def f_rho_w(TF,PP):
            return math.exp(  (c_f*(PP-pin)-b_f*(TF-tin)) )*rhoin
        ###################################
        gwb=9.80665                 #m/s^2
        ###################################
        nz=50
        h0=25.                       #m
        ####################################
        ####liquid
        pin=49.03325e6              #Pa
        tin=58.975 +273.15          #K
        qsc=1000.   /86400.          #m3/s
        ####################################
        nz=100
        h0=35
            
        tsc=15+273.15
        pin=45387e3 
        tin=140.69+273.15
        
        qsc=0.05
        wCO2=0
        
        U=0
        
            
        if wCO2!=0.:
            wCO2=0.
            # rhoin,cp,b_f,c_f,epsjt,mu=fco2(tin,pin,wCO2)
        else:
            iawb=iapws.IAPWS95(P=pin/1e6,T=tin) #MPa,K
            c_f=iawb.kappa/1e6
            b_f=iawb.alfav
            cp=iawb.cp*1e3
            rhoin=iawb.rho
            epsjt=iawb.joule/1e6
            mu=iawb.mu
           
        
        
        ###################################
        rw=.2302 
        
        dz=np.full(nz+1,h0)
        ###################################
        fdt=1e-0                    
        ndt=300                     
        tf=1* 86400                 
        
        
        maxdp_enabled=checkbox_maxAllowed.value            
        max_allowed_dp=100e3         
        max_allowed_dt=1
            
        #convergence tolerances
        tolp,tolt,tolq=1e-2,1e-2,1e-2
        ###################################
        tnow=np.logspace(np.log10(fdt), np.log10(tf), num=ndt)
        tnow=np.insert(tnow,0,0)
        ###################################
        ###################################
        # time step n+1, next iteration
        p=np.zeros(nz+2) 
        t=np.zeros(nz+2) 
        q=np.zeros(nz+1) #q[nz+1] does not exist!
        # time step n+1, previous iteration
        op=np.copy(p)
        oq=np.copy(q)
        ot=np.copy(t)
        # time step n
        p_=np.copy(p)
        q_=np.copy(q)
        t_=np.copy(t)
        ###################################
        global zb
        global z
        
        zb=np.zeros(nz+1)
        for i in range(1,nz+1):
            zb[i]=zb[i-1]+dz[i]
            
        z=np.zeros(nz+2)
        z[0]=0-dz[1]/2.
        for i in range(1,nz+1):
            z[i]=(zb[i]+zb[i-1])/2.
        z[nz+1]=zb[nz]+dz[nz]/2.
        ################################### 
        
        p[:]=pin
        
        t[:]=tin
        
        q[:]=0
        
        #############################
        if 1>2:
            fark=(tsc-t[0])/nz
            for j in range(1,nz+2):
                t[j]=t[j-1]+fark
        else:
            fark=(tsc-t[1])/nz
            for j in range(2,nz+2):
                t[j]=t[j-1]+fark
            t[0]=t[1]-fark
            t[0]=t[1]###ATASYON
        #############################
        tol=0.001
        for j in range(0,nz+1):
            dza=dz[j]
          
            p0=p[j]
            d0=f_rho_w(t[j],p[j])
            
            
            
            fark=1.0
            while fark>tol:
                p1 = p0 - gwb * dza * d0
                d1 = f_rho_w(t[j+1], p1) #review t[i] or t[i+1] ???
                
                d2 = (d0 + d1) / 2.0
                p2 = p0 - gwb * dza * d2
            
                fark = abs(p1 - p2)
                d0=d2
                
            p[j+1] = p2
        
        
        # p[0]=p[1]*1.
        
        tst=np.copy(t)
        pst=np.copy(p)
        
        # sys.exit()
        ###################################
        ###################################
        def lwb(j):
            return    3+(j-1)*3
        ####################################
        nbv=3               #BC_bottom  p,t,q
        nbv+=nz*3           #wellbore   p,t,q
        nbv+=2              #BC_top     p,t [no q]
        ####################################
        ####################################
        M=np.zeros((nbv,nbv))
        yy=np.zeros(nbv)
        ####################################
        def m(_r_,_c_,_d_):
            row.append(_r_)
            col.append(_c_)
            data.append(_d_)
        ####################################
        ###################################
        pj=[]
        tj=[]
        qj=[]
        for j in range(0,nz+2):
            pj.append([])
            tj.append([])
            if j!=nz+1: qj.append([]) 
        def loggb():
            for gj in range(0,nz+2):
                pj[gj].append(p[gj]/1000.)
                tj[gj].append(t[gj])
                if gj!=nz+1: qj[gj].append(q[gj])
        ###################################
        global pltp
        global pltt
        global pltq
        pltp,pltt,pltq=[],[],[]
        
        def logplt():    
            pltp.append(np.copy(p)/1000.)
            pltt.append(np.copy(t))
            pltq.append(np.copy(q))
        ###################################
            
        pt=[0.]
        logplt()
        loggb()
        
        
        ss=0
        while ss!=len(tnow)-1:
            ss+=1
            dt=tnow[ss]-tnow[ss-1]
                  
            op=np.copy(p)
            oq=np.copy(q)
            ot=np.copy(t)
            
            p_=np.copy(p)
            q_=np.copy(q)
            t_=np.copy(t)
        
            bok=1
            while bok>0:
                #bok=-1 #do not iterate!
               
                yy[:]=0.
                data, row, col =[],[],[]
                
                ####################################### entry BC
                j=lwb(0) ### BOTTOMHOLE
                
                ### PRESSURE
                m(j,j,  -1.)
                m(j,lwb(1), 1.)
                
                ### TEMPERATURE
                m(j+1,j+1,    1.)
                yy[j+1]= tst[0]
                
                ### RATE
                m(j+2,j+2,     1.)
                yy[j+2]=       qsc
                
                
                ####################################### exit BC
                j=lwb(nz+1)  ### WELLHEAD
                
                ### PRESSURE
                ##################### constant p [original]
                m(j,j,    1.)
                yy[j]= pst[-1] 
                #####################
                
                ### TEMPERATURE
                ##################### periodic t [original]
                m(j+1,j+1,           -1.)
                m(j+1,lwb(nz)+1,      2.)
                m(j+1,lwb(nz-1)+1,   -1.)
                
                #######################################
                for gb in range(1,nz+1):
                    j=lwb(gb)
                    
                    Aw=(rw**2.)*math.pi
                    D=2.*rw
                    
                    _z,z_=0,0
                    if j!=nz:  z_=z[gb+1]-z[gb]
                    if j!=1:    _z=z[gb]-z[gb-1]
                    #######################################
                    c1=dt/Aw
                    c2=dt*Aw
                    
                    
                    rhogb=f_rho_w(t[gb],p[gb])
                        
                        
                    vv= -b_f/c_f
                    zo= 1./(rhogb*cp)            
                    c4= -epsjt-zo
                
                    #######################################
                    ### MASS BALANCE
                    #######################################
                    #p - p
                    m(j,j,      1.)
                    yy[j]+=     1.* p_[gb]
                    
                    #p - t
                    m(j,j+1,    vv)
                    yy[j]+=     vv* t_[gb]
                    
                    #p - q
                    b=c1/c_f/dz[gb]  
                   
                    m(j,j+2,     b)
                    m(j,j+2-3,  -b)
                    #######################################
                    ### ENERGY BALANCE
                    #######################################
                    #t - t
                    m(j+1,j+1,      1.) 
                    yy[j+1]+=       1.* t_[gb]
                    #t - p
                    m(j+1,j,        c4)
                    yy[j+1]+=       c4* p_[gb]
                    ###########
        
                   
                    if U!=0:
                        st= dt*zo* 2.*U/rw
                        
                        #t - t
                        m(j+1,j+1,    st)           
                        yy[j+1]+=     st * tst[gb]       
                    
                    
                    if 1<2:
                        
                        b=c1*q[gb]/ dz[gb]
                        if q[gb]>0:
                            
                            # Mpp
                            m(j,j,              b)
                            # Mpt
                            m(j,j+1,            b*vv)
                            
                            # Mtt
                            m(j+1,j+1,          b)                    
                            # Mtp
                            m(j+1,j,            b*-epsjt)
                            
                        elif q[gb]<0:
                            
                            #mass
                            m(j,j+1  +3,           b*vv)
                            
                            #energy
                            m(j+1,j+1 +3,          b)
                            
                            #mass
                            m(j,j +3,              b)
                            
                            #energy
                            m(j+1,j  +3,           b*-epsjt)
                        
                        
                        
                        b=c1*q[gb-1]/ dz[gb]
                        if q[gb-1]>0:
                            
                            #mass
                            m(j,j+1  -3,           -b*vv)
                            
                            #energy
                            m(j+1,j+1  -3,         -b)
                            
                            #mass
                            m(j,j  -3,             -b)
                            
                            #energy
                            m(j+1,j  -3,           -b*-epsjt)
                            
                        elif q[gb-1]<0:
                            
                            #mass
                            m(j,j+1  ,         -b*vv)
                            
                            #energy
                            m(j+1,j+1,         -b)
                            
                            #mass
                            m(j,j ,            -b)
                            
                            #energy
                            m(j+1,j  ,         -b*-epsjt)
                        
                    
                   
                    
                    #######################################
                    ### MOMENTUM BALANCE
                    #######################################
                        
                    #q - q 
                    m(j+2,j+2,    1.)
                    yy[j+2]+=     1.* q_[gb] -Aw*dt*gwb     #gravity
                        
                    rhogb1=f_rho_w(t[gb+1],p[gb+1])
                    rhoave=(rhogb1+rhogb)/2.
                    
                    b=c2/rhoave /z_
                    m(j+2,j+3,    b)
                    m(j+2,j,     -b) 
                            
                 
                    
        
                  
        
                #######################################
                mtx = csr_matrix((data, (row, col)), shape=(nbv, nbv))
                x = linalg.spsolve(mtx, yy)
                #######################################
                sos=0
                
                p[0]=x[sos]
                sos+=1
                t[0]=x[sos]
                sos+=1
                q[0]=x[sos]
                sos+=1
                
                for gb in range(1,nz+1):
                    p[gb]=x[sos]
                    sos+=1
                    t[gb]=x[sos]
                    sos+=1
                    q[gb]=x[sos]
                    sos+=1
                    
                p[nz+1]=x[sos]
                sos+=1
                t[nz+1]=x[sos]
                sos+=1  
                
                
                if np.amin(p)<0. or np.amax(p)>pin*5.:
                    sys.exit('pres below zero or too high!')        
                if np.amin(t)<0. or np.amax(t)>tin*5.:
                    sys.exit('temp below zero or too high!')        
                #######################################
            
                if bok==1:    
                    maxp, maxq, maxt, maxdp, maxdt = 0.,0.,0.,0.,0.
                    for j in range(1,nz+1):
                        
                        temp1=abs((op[j]-p[j])/(p[j]+1e-13))
                        temp2=abs((oq[j]-q[j])/(q[j]+1e-13))
                        temp3=abs((ot[j]-t[j])/(t[j]+1e-13))
                        
                        if temp1>maxp: maxp=temp1
                        if temp2>maxq: maxq=temp2 
                        if temp3>maxt: maxt=temp3 
                        
                        temp4=abs(op[j]-p[j])           
                        if temp4>maxdp: maxdp=temp4
                        temp5=abs(ot[j]-t[j])           
                        if temp5>maxdt: maxdt=temp5
                    
                    if (ss>1 and maxdp_enabled==1) and (maxdp>max_allowed_dp or maxdt>max_allowed_dt):
                        if checkbox_log.value==True:print(tnow[ss],'not converged due to max allowed dp or dt=', maxdp, maxdt )
                        if checkbox_log.value==True:print(tnow[ss],'is not valid!')
                        
                        tnow=np.insert(tnow,ss,tnow[ss-1]+dt/2.)
                        
                        if checkbox_log.value==True:print('new time:',tnow[ss])                
                        dt=tnow[ss]-tnow[ss-1]
                        
                        p=np.copy(p_)
                        q=np.copy(q_)
                        t=np.copy(t_)
                        
                        op=np.copy(p_)
                        oq=np.copy(q_)
                        ot=np.copy(t_)
                        
                    else:
                        if maxp<=tolp and maxq<=tolq and maxt<=tolt: #tolerances    
                            bok=-1
                            if checkbox_log.value==True: print('t=',round(tnow[ss],3),'sec ********* , maxdp[kPa]:', maxdp, 'maxdt[K]:', maxdt)
                        else:
                            yaz=''
                            if not (maxp<= tolp): yaz+= '(p)'
                            if not (maxt<= tolt): yaz+= '(t)'
                            if not (maxq<= tolq): yaz+= '(q)'
                            if checkbox_log.value==True: print('not converged due to ', yaz)
                        
                            op=np.copy(p)
                            oq=np.copy(q)
                            ot=np.copy(t)
                    
                #######################################
            #######################################
            # pt.append(tnow[ss]/3600.) 
            pt.append(tnow[ss]/1.)  ### ATASYON: I DID IT.
            
            logplt()
            loggb()
            #######################################

        print('run succesfully completed')

In [282]:
def plot_calculation(button):
    with output:
        output.clear_output()

        plt.figure(figsize=(6.4*2,4.8*2))
        plt.subplot(131)
        plt.plot(pltq[0],zb,'-.',label='initial')
        plt.plot(zb,pltq[-1],zb,'x-',label='final')
        plt.ylim(0)
        plt.legend()
        plt.title('FLOW RATE')
        plt.subplot(132)
        plt.plot(pltp[0],z,'-.',label='initial')
        plt.plot(pltp[-1],z,'x-',label='final')
        plt.legend()
        plt.title('PRESSURE')
        plt.subplot(133)
        plt.plot(pltt[0],z,'-.',label='initial')
        plt.plot(pltt[-1],z,'x-',label='final')
        plt.legend()
        plt.title('TEMPERATURE')
        plt.show()

In [283]:
run_button.on_click(run_calculation)
plot_button.on_click(plot_calculation)

In [284]:
display(
    widgets.VBox([
        hl1,pin,tin,tsc,qsc,wCO2,
        hl2,rw,ang,nz,h0,U,
        hl3,fdt,ndt,tf,
        hl4,tolp,tolt,tolq,checkbox_maxAllowed,max_allowed_dp,max_allowed_dt,
        checkbox_log,
        run_button,plot_button,
        output
    ])
)